# Test for the GLM function subject_info

In [6]:
"""
FUNCTION TO GET THE SUBJECT-SPECIFIC INFORMATION
:param events: list with paths to events files
:param confounds: list with paths to confounds files
:return: Bunch object with event onsets, durations and regressors
"""

# import libraries (needed to be done in the function):
# import basic libraries:
import os
import glob
import sys
import warnings
from os.path import join as opj
# import nipype libraries:
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.utils.profiler import log_nodes_cb
from nipype import config, logging
# import spm and matlab interfaces:
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.interfaces.spm.model import (
    Level1Design, EstimateModel, EstimateContrast, ThresholdStatistics,
    Threshold)
from nipype.interfaces.matlab import MatlabCommand
from nipype.interfaces import spm
# import fsl interfaces:`
from nipype.workflows.fmri.fsl import create_susan_smooth
from nipype.interfaces.fsl.utils import ExtractROI
# import libraries for bids interaction:
from bids.layout import BIDSLayout
# import freesurfer interfaces:
# import custom functions:
from fMRIreplay_glm_functions import (
    get_subject_info, plot_stat_maps, leave_one_out)
import pandas as pd
from nipype.interfaces.base import Bunch

220623-17:22:17,378 nipype.interface INFO:
	 We advise you to upgrade DIPY version. This upgrade will open access to more function
220623-17:22:17,382 nipype.interface INFO:
	 We advise you to upgrade DIPY version. This upgrade will open access to more function
220623-17:22:17,384 nipype.interface INFO:
	 We advise you to upgrade DIPY version. This upgrade will open access to more models


/home/liulab/anaconda3/envs/huangqi_fmriprep/lib/python3.8/site-packages/niflow/nipype1/workflows/dmri/mrtrix/group_connectivity.py:16: UserWarning: cmp not installed
  warnings.warn('cmp not installed')
/home/liulab/anaconda3/envs/huangqi_fmriprep/lib/python3.8/site-packages/nipype/workflows/__init__.py:28: UserWarning: Nipype 1 workflows have been moved to the niflow-nipype1-workflows package. nipype.workflows.* provides a reference for backwards compatibility. Please use niflow.nipype1.workflows.* to avoid this warning.
  warnings.warn(" ".join(_msg))


In [7]:
# time of repetition, in seconds:
time_repetition = 1.3
# total number of runs:
num_runs = 2
# total number of folds for leave-one-out-run:
num_fold = 4
# smoothing kernel, in mm:
fwhm = 4
# number of dummy variables to remove from each run:
num_dummy = 0

In [8]:
project = 'fMRIreplay'
# initialize empty paths:
path_root = None
sub_list = None
# path to the project root:
project_name = 'fmrireplay-glm'
path_root = opj(os.getcwd().split(project)[0] ,'fMRIreplay_hq')
path_bids = opj(path_root ,'fmrireplay-bids','BIDS')
path_fmriprep = opj(path_bids ,'derivatives')
path_glm = opj(path_root, project_name)

In [10]:
sub_list = ['sub-04']
templates = dict(
    confounds=opj(path_fmriprep,'{subject_id}' , 'func','*vfl*confounds_timeseries.tsv'),
    events=opj(path_bids,'{subject_id}', 'func','*vfl*events.tsv'),
    func=opj(path_fmriprep,'{subject_id}', 'func','*vfl*space-T1w*preproc_bold.nii.gz'),
    anat=opj(path_fmriprep,'{subject_id}', 'anat','{subject_id}_desc-preproc_T1w.nii.gz'),
    wholemask=opj(path_fmriprep,'{subject_id}','func','*vfl*space-T1w*brain_mask.nii.gz'),
)
# define the selectfiles node:
selectfiles = Node(SelectFiles(templates, sort_filelist=True),
                   name='selectfiles')
# set expected thread and memory usage for the node:
selectfiles.interface.num_threads = 1
selectfiles.interface.mem_gb = 0.1
selectfiles.inputs.subject_id = 'sub-04'
selectfiles_results = selectfiles.run()

220623-17:22:28,418 nipype.workflow INFO:
	 [Node] Setting-up "selectfiles" in "/tmp/tmpmpnb5mat/selectfiles".
220623-17:22:28,421 nipype.workflow INFO:
	 [Node] Executing "selectfiles" <nipype.interfaces.io.SelectFiles>
220623-17:22:28,424 nipype.workflow INFO:
	 [Node] Finished "selectfiles", elapsed time 0.002255s.


In [11]:
# event types we consider:
vfl_event_spec = {
    'correct_rejection': {'Ori': 0, 'accuracy': 1},
    'hit': {'Ori': 180, 'accuracy': 1},
    'false_alarm': {'Ori': 0, 'accuracy': 0},
    'miss': {'Ori': 180, 'accuracy': 0},
}

In [13]:
# read the events and confounds files of the current run:
events = selectfiles_results.outputs.events[0]
confounds = selectfiles_results.outputs.confounds[0]

In [14]:
#event_names = ['correct_rejection']


run_events = pd.read_csv(events, sep="\t")
run_confounds = pd.read_csv(confounds, sep="\t")

# define confounds to include as regressors:
confounds = ['trans', 'rot', 'a_comp_cor', 'framewise_displacement']
# search for confounds of interest in the confounds data frame:
regressor_names = [col for col in run_confounds.columns if
                   any([conf in col for conf in confounds])]

In [15]:
def replace_nan(regressor_values):
    # calculate the mean value of the regressor:
    mean_value = regressor_values.mean(skipna=True)
    # replace all values containing nan with the mean value:
    regressor_values[regressor_values.isnull()] = mean_value
    # return list of the regressor values:
    return list(regressor_values)

# create a nested list with regressor values
regressors = [replace_nan(run_confounds[conf]) for conf in regressor_names]

onsets = []
durations = []
event_names = []

In [16]:
for event in vfl_event_spec:
    onset_list = list(
        run_events['onset']
        [(run_events['task'] == 'VFL') &
         (run_events['Ori'] == vfl_event_spec[event]['Ori']) &
         (run_events['accuracy'] == vfl_event_spec[event]['accuracy'])])
    duration_list = list(
        run_events['duration']
        [(run_events['task'] == 'VFL') &
         (run_events['Ori'] == vfl_event_spec[event]['Ori']) &
         (run_events['accuracy'] == vfl_event_spec[event]['accuracy'])])
    if (onset_list != []) & (duration_list != []):
        event_names.append(event)
        onsets.append(onset_list)
        durations.append(duration_list)

In [22]:
# durations

In [23]:
# onsets

In [19]:
# create a bunch for each run:
subject_info = Bunch(
    conditions=event_names, onsets=onsets, durations=durations,
    regressor_names=regressor_names, regressors=regressors)

In [24]:
# subject_info

In [20]:
sorted(event_names)

['correct_rejection', 'false_alarm', 'hit', 'miss']

In [ ]:
# def leave_one_out(subject_info, event_names, data_func, run):
    """
    Select subsets of lists in leave-one-out fashion:
    :param subject_info: list of subject info bunch objects
    :param data_func: list of functional runs
    :param run: current run
    :return: return list of subject info and data excluding the current run
    """
subject_info = 
event_names = 
data_func = 
run = 

# create new list with event_names of all runs except current run:
for n in nums_fold:
    if n = 1:
        event_names = [info for i, info in [0:int(enumerate(event_names)/2-1)] if i != fold]
        num_events = [len(i) in event_names/2]
        max_events = event_names[num_events.index(max(num_events))]/2
    elif n = 2:
        event_names = [info for i, info in [int(enumerate(event_names))/2:int(enumerate(event_names))] if i != fold]
        num_events = [len(i) in event_names/2]
        max_events = event_names[num_events.index(max(num_events))]/2

        
event_names = [info for i, info in enumerate(event_names) if i != fold]
num_events = [len(i) for i in event_names]
max_events = event_names[num_events.index(max(num_events))]

# create list of contrasts:
stim = 'correct_rejection'
contrast1 = (stim, 'T', max_events, [1 if stim in s else 0 for s in max_events])
contrasts = [contrast1]

# create new list with subject info of all runs except current run:
subject_info = [info for i, info in enumerate(subject_info) if i != run]
# create new list with functional data of all runs except current run:
data_func = [info for i, info in enumerate(data_func) if i != run]

# return the new lists
return subject_info, data_func, contrasts


In [ ]:

"""
Select subsets of lists in leave-one-out fashion:
:param subject_info: list of subject info bunch objects
:param data_func: list of functional runs
:param run: current run
:return: return list of subject info and data excluding the current run
"""
subject_info = subject_info
event_names = event_names
data_func = 
run = nums_run
# create new list with event_names of all runs except current run:
event_names = [info for i, info in enumerate(event_names) if i != run]
num_events = [len(i) for i in event_names]
max_events = event_names[num_events.index(max(num_events))]

# create list of contrasts:
stim = 'correct_rejection'
contrast1 = (stim, 'T', max_events, [1 if stim in s else 0 for s in max_events])
contrasts = [contrast1]

# create new list with subject info of all runs except current run:
subject_info = [info for i, info in enumerate(subject_info) if i != run]
# create new list with functional data of all runs except current run:
data_func = [info for i, info in enumerate(data_func) if i != run]

# return the new lists
return subject_info, data_func, contrasts